In [41]:
from google.colab import drive
drive_path = '/content/drive'
drive.mount(drive_path)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [42]:
import json

secret_file_path = "/content/drive/My Drive/Secret_Keys/OpenAI_Secret_Key.json"

with open(secret_file_path, "r") as f:
  data=json.load(f)

secret_Key = data["open_ai_key"]

In [43]:
def get_game_state():
  print("Enter current game state value (0-100): \n ")

  while True:
    try:
      health = float(input("Player health (0-100):  "))
      enemy_distance = float(input("Enemy Distance (0-100): "))
      resources = float(input("Available Resources (0-100): "))

      # all is inbuilt function
      if all(0 <= val <=100 for val in [health,enemy_distance,resources]):
        return health,enemy_distance,resources
      else:
        print("All values must be 0 and 100")
    except ValueError:
        print("Please enter valid numeric input")

In [44]:
def compute_utilities(health,enemy_distance,resources):
    utilities ={}
    # Attack: High utility if health is good, enemy is close, and resources are available
    utilities["attack"] = (
        0.4 * (100 - enemy_distance) +
        0.3 * health +
        0.3 * resources
    )

    # Defend: Higher utility when health is moderate and enemy is close
    utilities["defend"] = (
        0.4 * (100 - health) +
        0.4 * (100 - enemy_distance) +
        0.2 * resources
    )

    # Retreat: Higher utility when health is low and enemy is close
    utilities["retreat"] = (
        0.5 * (100 - health) +
        0.4 * (100 - enemy_distance) +
        0.1 * (100 - resources)
    )

    return utilities

#Pick the best action
def select_best_action(utilities):
    print("\nComputed Utility Scores:")
    for action, score in utilities.items():
        print(f"{action.ljust(10)} -> {score:.2f}")

    best = max(utilities, key=utilities.get)
    print(f"\nSelected Action: {best}")
    return best

In [47]:
# Ask GPT-4o-mini to explain the selected action
def explain_action_with_openai(action, health, enemy_distance, resources):
    from openai import OpenAI
    client = OpenAI(api_key=secret_Key)  # Replace with your key

    prompt = (
        f"As a game AI, I chose to '{action}' based on the following:\n"
        f"- Player health: {health}\n"
        f"- Enemy distance: {enemy_distance}\n"
        f"- Available resources: {resources}\n"
        f"\nExplain why this was the best decision."
    )

    response = client.chat.completions.create(
        model="gpt-3.5-turbo",  # <-- changed from gpt-4o-mini to gpt-3.5-turbo
        messages=[
            {"role": "system", "content": "You are a game AI coach that explains decisions clearly."},
            {"role": "user", "content": prompt}
        ]
    )
    return response.choices[0].message.content.strip()

In [48]:
#Run everything
def run_game_ai_agent():
    print("=== Game AI: Utility-Based Combat Agent ===\n")
    health, enemy_distance, resources = get_game_state()
    utilities = compute_utilities(health, enemy_distance, resources)
    best_action = select_best_action(utilities)

    use_openai = True
    if use_openai:
        print("\nExplanation from GPT-4o-mini:\n")
        explanation = explain_action_with_openai(best_action, health, enemy_distance, resources)
        print(explanation)

# Start the agent
run_game_ai_agent()


=== Game AI: Utility-Based Combat Agent ===

Enter current game state value (0-100): 
 
Player health (0-100):  10
Enemy Distance (0-100): 55
Available Resources (0-100): 85

Computed Utility Scores:
attack     -> 46.50
defend     -> 71.00
retreat    -> 64.50

Selected Action: defend

Explanation from GPT-4o-mini:

Based on the information provided, choosing to defend was the best decision for the following reasons:

1. Player health: With the player's health at 10.0, it suggests that the player may be at a vulnerable state and could benefit from taking a defensive stance to avoid potential damage from the enemy.

2. Enemy distance: The enemy being at a distance of 55.0 suggests that they are not in close proximity, which provides an opportunity for the player to defend and strategize their next move without being immediately engaged in combat.

3. Available resources: Having 85.0 resources available indicates that the player has the capability to strengthen their defensive position or